In [1]:
# Initial setup following http://docs.chainer.org/en/stable/tutorial/basic.html
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
import chainer.dataset
import chainer.datasets

## Built-in dataset modules

Some dataset format is already implemented in `chainer.datasets`

### TupleDataset

### DictDataset

TBD

### DictDataset

TBD

## Implement your own custom dataset

You can define your own dataset by implementing a sub class of `DatasetMixin` in `chainer.dataset`

### ImageDataset

TBD

### LabeledImageDataset

TBD

### SubDataset

TBD

It can be used for cross validation.

## Implement your own custom dataset

You can define your own dataset by implementing a sub class of `DatasetMixin` in `chainer.dataset`

## DatasetMixin